# spark-df-profiling Meteorites example

Source of data: https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh

I have previously transformed the downloaded csv to a [Parquet](https://parquet.apache.org/) table, but that doesn't matter. As long as you have your Spark Dataframe loaded, you are good to go.

Note that I had to directly pull the latest version of spark_df_profiling (testing required?) from the GitHub repo as recent changes to Pandas have broken it. e.g. .ix changed to .iloc

## Import library

In [1]:
from pyspark.sql import SparkSession 

import spark_df_profiling


In [2]:
spark = SparkSession.builder.master("local").getOrCreate()

22/09/26 12:17:45 WARN Utils: Your hostname, tmps-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.20.131.88 instead (on interface en0)
22/09/26 12:17:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/26 12:17:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/26 12:17:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/26 12:17:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
type(spark)

pyspark.sql.session.SparkSession

In [5]:
# sqlContext = SQLContext(spark) -- this doesn't work - deprecated?

## Create the DataFrame

In [6]:
# df = sqlContext.read.parquet("/Users/Julio/Downloads/Meteorite_Landings.parquet").cache()

df = spark.read.csv("../data/Meteorite_Landings.csv", header=True).cache()

In [7]:
df

DataFrame[name: string, id: string, nametype: string, recclass: string, mass (g): string, fall: string, year: string, reclat: string, reclong: string, GeoLocation: string]

In [8]:
df.columns

['name',
 'id',
 'nametype',
 'recclass',
 'mass (g)',
 'fall',
 'year',
 'reclat',
 'reclong',
 'GeoLocation']

Spark Dataframes have the built-in method `.describe()`. Let's see what it shows:

In [9]:
df.describe().show()

+-------+--------+-----------------+--------+-----------+------------------+-----+------------------+------------------+-----------------+--------------------+
|summary|    name|               id|nametype|   recclass|          mass (g)| fall|              year|            reclat|          reclong|         GeoLocation|
+-------+--------+-----------------+--------+-----------+------------------+-----+------------------+------------------+-----------------+--------------------+
|  count|   45716|            45716|   45716|      45716|             45585|45716|             45425|             38401|            38401|               38401|
|   mean|    null|26889.73510368361|    null|       null|13278.078548601488| null|1991.8288167308751|-39.12258031142524|61.07431852625881|                null|
| stddev|    null|16860.68303027627|    null|       null| 574988.8764104763| null|25.052766117703865| 46.37851135669301|80.64729807908495|                null|
|    min|  Aachen|                1|  Re

## Generate the report

Now let's use `spark_df_profiling`:

In [10]:
report = spark_df_profiling.ProfileReport(df)

/Users/mjboothaus/code/github/mjboothaus/sparky/.venv_dev_trial-spark-local-pandas-api/lib/python3.9/site-packages/spark_df_profiling/base.py:416: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(describe_unique_1d(df, column))
/Users/mjboothaus/code/github/mjboothaus/sparky/.venv_dev_trial-spark-local-pandas-api/lib/python3.9/site-packages/spark_df_profiling/base.py:416: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(describe_unique_1d(df, column))
/Users/mjboothaus/code/github/mjboothaus/sparky/.venv_dev_trial-spark-local-pandas-api/lib/python3.9/site-packages/spark_df_profiling/base.py:418: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(describe_categoric

In [11]:
report

## Save report to file

In [12]:
report.to_file("../data/meteor_profile.html")

In [13]:
type(report)

spark_df_profiling.ProfileReport

22/09/26 13:24:29 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 971822 ms exceeds timeout 120000 ms
22/09/26 13:24:29 WARN SparkContext: Killing executors is not supported by current scheduler.
